**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

In [ ]:
%pip install -q ipylab

# Plugins

We use the [pluggy](https://pluggy.readthedocs.io/en/stable/index.html#pluggy) plugin system.

Plugins can be either:
* Manually registered with the `plugin_manager` or
* defined as an `entrypoint` for modules.

The advantage of using entry points is that the plugin is registered automatically and can run in the always running `iyplab` kernel. But requires the extra effort installing a module with the defined entry point.

The following plugins (*hookspecs*) are available.

In [ ]:
# Existing hook specs
from IPython import display as ipd

import ipylab
import ipylab.hookspecs

app = ipylab.App()
out = ipylab.SimpleOutput(layout={"height": "300px"}).add_class("ipylab-ResizeBox")
out.push(ipd.Markdown("## Plugins\n\nThe following plugins (*hookspecs*) are available."))
for n in dir(ipylab.hookspecs):
    f = getattr(ipylab.hookspecs, n)
    if not hasattr(f, "ipylab_spec"):
        continue
    out.push(ipd.Markdown(f"### `{f.__name__}`"), ipd.Markdown(f.__doc__))
out

## Autostart

The `autostart` hookspec is called as when the App is ready.

Possible uses include:
* Create and register custom commands;
* Create launchers;
* Modify the appearance of Jupyterlab.

## Example launching a small app

The following example demonstrates a few plugins:
1. `autostart` This is a 'historic' plugin and runs once the app is 'ready'. 
2. `default_namespace_objects` This allows for modify the namespace feature implemented in ipylab.
3. `ready` This hook is called for each Ipylab instance when it is ready. 

In [ ]:
# @myproject.pluginmodule.py


async def create_app(app: ipylab.App, cwd="", extra=""):
    # The code in this function is called in the new kernel.
    # Ensure imports are performed inside the function.
    import ipywidgets as ipw
    from IPython import display as ipd

    import ipylab

    md = f"""
## test
Welcome to my app.

## cwd

The folder in Jupyter tree when opened
`{cwd=}`

### extra

Extra args provided to the launcher
`{extra=}`

### Context menu
Try the context menu with: right click -> Open console

The shell connection for the widget is added to the console namespace as `ref`. The widget of the shell connection can be access from the shell connection as `ref.widget`.
```
    """

    panel = ipylab.Panel()
    panel.title.label = app.vpath
    notify_type = ipw.Dropdown(description="Notify type", options=ipylab.NotificationType)
    notify_message = ipw.Combobox(placeholder="Enter message")
    notify_button = ipw.Button(description="Notify")
    notify_button.on_click(lambda _: app.start_coro(app.notification.notify(notify_message.value, notify_type.value)))  # type: ignore
    box = ipw.HBox([notify_type, notify_message, notify_button], layout={"align_content": "center", "flex": "1 0 auto"})

    out = ipw.Output()
    out.append_display_data(ipd.Markdown(md))
    panel.children = [out, box]

    # Do something when the window is closed (shutdown the kernel)
    def observe_connections(change):
        if not change["new"] and panel.comm:
            panel.close()

            async def shutdown():
                result = await app.dialog.show_dialog("Shutdown kernel?")
                if result["value"]:
                    await app.notification.notify("Shutting down kernel", type=ipylab.NotificationType.info)
                    await app.shutdown_kernel()

            app.start_coro(shutdown())

    # Add a plugin in this kernel. Instead of defining a class, you can also define a module eg: 'ipylab.lib.py'
    class MyLocalPlugin:
        @ipylab.hookimpl
        def default_namespace_objects(self, namespace_id: str, app: ipylab.App):
            if namespace_id == "test":
                # Define alternate default objects for this namespace
                return {"test": "TEST", "app": app, "ipylab": ipylab}
            return {}

    ipylab.plugin_manager.register(MyLocalPlugin())

    app.shell.observe(observe_connections, "connections")
    return panel


import ipylab  # noqa: E402


class MyPlugins:
    @ipylab.hookimpl(specname="autostart")
    async def register_commands(self, app: ipylab.App):
        cmd = await app.commands.add_command(
            "Start my app",
            execute=app.shell.add,
            args={
                "obj": create_app,
                "vpath": {"title": "Virtual path for app"},
            },
            label="Start",
            caption="Start my custom app",
            icon_class="jp-PythonIcon",
        )
        await app.launcher.add(cmd, "Ipylab", extra="Ipylab extra arguments")

    @ipylab.hookimpl
    def vpath_getter(self, app: ipylab.App, kwgs: dict):  # noqa: ARG002
        # if the ipylab kernel is running the per-kernel widget manager
        if app.per_kernel_widget_manager_detected:
            return None
        return app.vpath


pluginmodule = MyPlugins()

In [ ]:
# Register the plugin
ipylab.plugin_manager.register(pluginmodule, "demo plugin")
ipylab.SimpleOutput(layout={"height": "200px"}).add_class("ipylab-ResizeBox").push(app.commands.all_commands)

In [ ]:
# Show the button in the launcher
await app.commands.execute("launcher:create")

## Entry points

Entry points provide for automatic registration for when ipylab is imported.

Add the following in your `pyproject.toml`

``` toml
[project.entry-points.ipylab]
myproject = "myproject.pluginmodule"
```

`pluginmodule` is the name of a python file (module) with the '.py' omitted.

It is also possible to define specific objects in the module to load by adding a ':' between the module name and the object. The object should be the name of an instance of a class in the module that defines the plugins.

``` toml
[project.entry-points.ipylab]
myproject1 = "myproject.pluginmodule:plugin1"
myproject2 = "myproject.pluginmodule:plugin2"

# in pluginmodule:

plugin1 = MyPlugins()
plugin2 = MyOtherPlugins()
```


### Ipylab kernel (autostart)

**Starting kernels with widgets enabled requires a widget manager capable of running independent of a notebook or console.**

Currently this isn't supported by IpyWidgets. There is an outstanding PR [here](https://github.com/jupyter-widgets/ipywidgets/pull/3922) which enables this. 

#### Command

The ipylab kernel can be started/re-started with the command 'Start ipylab kernel' (`Ctrl c` -> 'Start or restart ipylab kernel').

#### Configure

An *autostart* ipylab kernel can be enabled in the configuration settings which will automatically start when Jupyterlab is started (intended to run locally). This provides a means for entry points to be loaded when Jupyterlab is launched.

To change the configuration use the Jupyterlab settings editor: 

`Ctrl + ,` 

`Ipylab` 

`ipylab kernel enabled = True`.

The next time Jupyterlab is started, a kerenel on the path 'ipylab' will be started.